##Init()

In [ ]:

import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))
torch.cuda.empty_cache()

!pip install farm==0.4.8 -q
# !pip install farm -q
!pip install urllib3==1.25.11 -q
# 0.4.7 has problem 
# !git clone https://github.com/deepset-ai/FARM.git
# !cd FARM && pip install -r requirements.txt && pip install --editable .


Torch 1.7.0+cu101 CUDA 10.1
Device: cuda:0
     |████████████████████████████████| 194kB 11.8MB/s 
     |████████████████████████████████| 748.8MB 16kB/s 
     |████████████████████████████████| 133kB 60.4MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
     |████████████████████████████████| 2.5MB 59.8MB/s 
     |████████████████████████████████| 47.7MB 143kB/s 
     |████████████████████████████████| 327kB 58.8MB/s 
     |████████████████████████████████| 7.2MB 46.1MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
     |████████████████████████████████| 3.0MB 46.3MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 133kB 10.5MB/s 
     |████████████████████████████████| 153kB 55.8MB/s 
     |████████████████████████████████| 163kB 57.5MB/s 
     |████

In [ ]:
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead,MultiLabelTextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
# from farm.conversion.transformers import Converter
# from farm.conversion.transformers import Converter
# import farm.conversion.transformers.Converter
from farm.train import Trainer,EarlyStopping
from farm.utils import MLFlowLogger
from farm.infer import Inferencer
from farm.evaluation.metrics import register_metrics
print("Hi")

Hi


##Multi-class classification on all 

In [ ]:

LABEL_LIST = ["hate", "fake","defamation","offensive","non-hostile"]
EMBEDS_DROPOUT_PROB = 0.2
LEARNING_RATE = 2e-6    #change to 5
N_EPOCHS = 50
N_GPU = 5
evaluate_every = 100
patience=7
from farm.evaluation.metrics import register_metrics
from sklearn.metrics import f1_score

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score


def f1_weighted(preds, labels):
    labels = labels[:,:-1]
    preds = preds[:,:-1]

    fine_f1          = f1_score(y_true=labels, y_pred=preds, average=None,zero_division=1)
    defame_f1        = fine_f1[0]
    fake_f1          = fine_f1[1]
    hate_f1          = fine_f1[2]
    offensive_f1     = fine_f1[3]
    print("coarse F1-score:     ", fine_f1)
    print("Defamation F1-score:     ", defame_f1)
    print("Fake F1-score:           ", fake_f1)
    print("Hate F1-score:           ", hate_f1)
    print("Offensive F1-score:      ", offensive_f1)
    weighted_fine_f1 = f1_score(y_true=labels, y_pred=preds, average='weighted',zero_division=1)
    print("Fine Grained F1-score:   ",weighted_fine_f1)
    
    return {"f1_weighted": weighted_fine_f1}

register_metrics(name="f1_weighted", implementation=f1_weighted)



In [ ]:

model_name = "xlm-roberta-large"       #"bert-base-german-cased"#"xlm-roberta-large"#"bert-large-cased"#"xlm-roberta-large"#"bert-large-uncased"#"xlm-roberta-base"#"bert-large-cased"
for i in range(1):   
  tokenizer = Tokenizer.load(
    pretrained_model_name_or_path= model_name,  #"xlm-roberta-base",
    do_lower_case=False)

  random_state = 2
  processor = TextClassificationProcessor(tokenizer=tokenizer,#delimiter=',',   
                                        max_seq_len=90,   #change ?
                                        data_dir="/content/drive/My Drive/constraint2021/data/siva/",
                                        train_filename="farm_cleaned_t2_consthinditrain.tsv",       #does it accept xlsx ? or convert to cdv ?
                                        dev_filename='farm_cleaned_t2_consthindivalid.tsv',
                                        test_filename=None,#"farm_uncleaned_test.tsv",   # HAVING NONE HERE IS VERY IMPORTANT .. o/w it will load from test.tsv and in your case, it has wrong columns 
                                        # dev_split=0.075,      #for 0.1, 1400/5400 goes away 
                                        label_list=LABEL_LIST,
                                        multilabel=True,
                                        metric="f1_weighted",
                                        label_column_name="multi_label")
  BATCH_SIZE = 16#32#3#2

  data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

  print("Seed set is {}".format(random_state))
  torch.manual_seed(random_state)
  prediction_head = MultiLabelTextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"),pred_threshold=0.4)

  MODEL_NAME_OR_PATH =  model_name   #"xlm-roberta-base"     #"bert-base-multilingual-uncased"
  language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

  # language_model = AdaptiveModel.convert_from_transformers('xlnet-base-cased', device=torch.device('cuda:0'), task_type="text_classification")
  model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=torch.device('cuda:0'))



  model, optimizer, lr_schedule = initialize_optimizer(
      model=model,
      device=torch.device('cuda:0'),
      learning_rate=LEARNING_RATE,
      n_batches=len(data_silo.loaders["train"]),
      n_epochs=N_EPOCHS)



  # DEfine Early Stopping:
  earlystopping = EarlyStopping(metric='f1_weighted',
                                save_dir='/content/drive/My Drive/constraint2021/data/siva/saved_models/final_multi_label_single_model',
                                patience=patience,
                                mode='max'  #  telling it to maximize the f1-score/metric. I f loss was here, min will be the mode
                                )
  # You can also load from early stopping best model as follows : model = Inferencer.load(earlystopping.save_dir)
  #  
  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      data_silo=data_silo,
      epochs=N_EPOCHS,
      n_gpu=N_GPU,
      evaluate_every = evaluate_every,
      early_stopping=earlystopping,
      lr_schedule=lr_schedule,
      device=torch.device('cuda:0'),
      disable_tqdm=True,from_step=0
  )




    


12/24/2020 08:36:06 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'XLMRobertaTokenizer'
12/24/2020 08:36:06 - INFO - filelock -   Lock 140521928288912 acquired on /root/.cache/torch/transformers/f7e58cf8eef122765ff522a4c7c0805d2fe8871ec58dcb13d0c2764ea3e4a0f3.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed.lock


12/24/2020 08:36:07 - INFO - filelock -   Lock 140521928288912 released on /root/.cache/torch/transformers/f7e58cf8eef122765ff522a4c7c0805d2fe8871ec58dcb13d0c2764ea3e4a0f3.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed.lock


12/24/2020 08:36:07 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
12/24/2020 08:36:07 - INFO - farm.data_handler.data_silo -   Loading train set from: /content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t2_consthinditrain.tsv 
12/24/2020 08:36:07 - INFO - farm.data_handler.utils -    Couldn't find /content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t2_consthinditrain.tsv locally. Trying to download ...
12/24/2020 08:36:07 - INFO - farm.data_handler.utils -   downloading and extracting file siva to dir /content/drive/My Drive/constraint2021/data
12/24/2020 08:36:07 - ERROR - farm.data_handler.utils -   Cannot download siva. Unknown data source.


FileNotFoundError: ignored

In [ ]:
trainer = Trainer(
      model=model,
      optimizer=optimizer,
      data_silo=data_silo,
      epochs=20,#N_EPOCHS,
      n_gpu=N_GPU,
      evaluate_every = evaluate_every,
      # early_stopping=earlystopping,
      lr_schedule=lr_schedule,
      device=torch.device('cuda:0'),
      disable_tqdm=True,from_step=0
  )


In [ ]:
model = trainer.train()
# f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
# f.write("\n Completed "+str(model_name)+"\n")
# f.close()
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
#[0 0 1 1 1] [0 0 1 0 0] non-hostile at end  MAY BE WROING  

12/24/2020 08:32:41 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `



RuntimeError: ignored

##Hierarchial multi-class classification (w/o non-hostile)

In [ ]:
#FOR FINAL 25% THRSHOLD IS SET 
LABEL_LIST = ["hate", "fake","defamation","offensive"]
EMBEDS_DROPOUT_PROB = 0.2
LEARNING_RATE = 2e-6    #change to 5
N_EPOCHS = 50
N_GPU = 5
evaluate_every = 100
patience=7
from farm.evaluation.metrics import register_metrics
from sklearn.metrics import f1_score

# def f1_weighted(preds, labels):
#     f1_weighted = f1_score(y_true=labels, y_pred=preds, average="weighted",zero_division=1)
#     return {"f1_weighted": f1_weighted}

import numpy as np
import pandas as pd

def f1_weighted(preds, labels):
    fine_f1          = f1_score(y_true=labels, y_pred=preds, average=None)
    hate_f1        = fine_f1[0]
    fake_f1          = fine_f1[1]
    defame_f1          = fine_f1[2]
    offensive_f1     = fine_f1[3]
    print("Defamation F1-score:     ", defame_f1)
    print("Fake F1-score:           ", fake_f1)
    print("Hate F1-score:           ", hate_f1)
    print("Offensive F1-score:      ", offensive_f1)
    weighted_fine_f1 = f1_score(y_true=labels, y_pred=preds, average='weighted')
    print("Fine Grained F1-score:   ",weighted_fine_f1)
    
    return {"f1_weighted": weighted_fine_f1}

register_metrics(name="f1_weighted", implementation=f1_weighted)



In [ ]:

model_name = "xlm-roberta-large"       #"bert-base-german-cased"#"xlm-roberta-large"#"bert-large-cased"#"xlm-roberta-large"#"bert-large-uncased"#"xlm-roberta-base"#"bert-large-cased"
for i in range(1):   
  tokenizer = Tokenizer.load(
    pretrained_model_name_or_path= model_name,  #"xlm-roberta-base",
    do_lower_case=False)

  random_state = 42
  processor = TextClassificationProcessor(tokenizer=tokenizer,#delimiter=',',   
                                        max_seq_len=90,   #change ?
                                        data_dir="/content/drive/My Drive/constraint2021/data/siva/",
                                        train_filename="farm_cleaned_t2_consthinditrain_wo_non_hostile.tsv",       #does it accept xlsx ? or convert to cdv ?
                                        dev_filename='farm_cleaned_t2_consthindivalid_wo_non_hostile.tsv',
                                        test_filename=None,#"farm_cleaned_t2_consthindivalid_wo_non_hostile.tsv",   # HAVING NONE HERE IS VERY IMPORTANT .. o/w it will load from test.tsv and in your case, it has wrong columns 
                                        # dev_split=0.075,      #for 0.1, 1400/5400 goes away 
                                        label_list=LABEL_LIST,
                                        multilabel=True,
                                        metric="f1_weighted",
                                        label_column_name="multi_label")
  BATCH_SIZE = 16#32#3#2

  data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

  print("Seed set is {}".format(random_state))
  torch.manual_seed(random_state)
  prediction_head = MultiLabelTextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"),pred_threshold=0.25)

  MODEL_NAME_OR_PATH =  model_name   #"xlm-roberta-base"     #"bert-base-multilingual-uncased"
  language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

  # language_model = AdaptiveModel.convert_from_transformers('xlnet-base-cased', device=torch.device('cuda:0'), task_type="text_classification")
  model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=torch.device('cuda:0'))



  model, optimizer, lr_schedule = initialize_optimizer(
      model=model,
      device=torch.device('cuda:0'),
      learning_rate=LEARNING_RATE,
      n_batches=len(data_silo.loaders["train"]),
      n_epochs=N_EPOCHS)



  # DEfine Early Stopping:
  earlystopping = EarlyStopping(metric='f1_weighted',
                                save_dir='/content/drive/My Drive/constraint2021/data/siva/saved_models/final_hierarchial_classification_model_xlmr_large_finetuned',#+str(random_state)+'-'+str(model_name)+'-2e-5-50epochs_patience7'
                                patience=patience,
                                mode='max'  #  telling it to maximize the f1-score/metric. I f loss was here, min will be the mode
                                )
  # You can also load from early stopping best model as follows : model = Inferencer.load(earlystopping.save_dir)
  #  
  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      data_silo=data_silo,
      epochs=N_EPOCHS,
      n_gpu=N_GPU,
      evaluate_every = evaluate_every,
      early_stopping=earlystopping,
      lr_schedule=lr_schedule,
      device=torch.device('cuda:0'),
      disable_tqdm=True
  )




    


12/06/2020 04:58:25 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'XLMRobertaTokenizer'
12/06/2020 04:58:25 - INFO - filelock -   Lock 139764560614176 acquired on /root/.cache/torch/transformers/f7e58cf8eef122765ff522a4c7c0805d2fe8871ec58dcb13d0c2764ea3e4a0f3.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed.lock


12/06/2020 04:58:26 - INFO - filelock -   Lock 139764560614176 released on /root/.cache/torch/transformers/f7e58cf8eef122765ff522a4c7c0805d2fe8871ec58dcb13d0c2764ea3e4a0f3.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed.lock


12/06/2020 04:58:26 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
12/06/2020 04:58:26 - INFO - farm.data_handler.data_silo -   Loading train set from: /content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t2_consthinditrain_wo_non_hostile.tsv 
12/06/2020 04:58:30 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 2678 dictionaries to pytorch datasets (chunksize = 536)...
12/06/2020 04:58:30 - INFO - farm.data_handler.data_silo -    0 
12/06/2020 04:58:30 - INFO - farm.data_handler.data_silo -   /w\
12/06/2020 04:58:30 - INFO - farm.data_handler.data_silo -   /'\
12/06/2020 04:58:30 - INFO - farm.data_handler.data_silo -   
Preprocessing Dataset /content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t2_consthinditrain_wo_non_hostile.tsv:   0%|          | 0/2678 [00:00<?, ?

Seed set is 42


12/06/2020 04:58:40 - INFO - filelock -   Lock 139763438434288 released on /root/.cache/torch/transformers/5ac6d3984e5ca7c5227e4821c65d341900125db538c5f09a1ead14f380def4a7.aa59609b4f56f82fa7699f0d47997566ccc4cf07e484f3a7bc883bd7c5a34488.lock


12/06/2020 04:58:41 - INFO - filelock -   Lock 139763438435184 acquired on /root/.cache/torch/transformers/a89d1c4637c1ea5ecd460c2a7c06a03acc9a961fc8c59aa2dd76d8a7f1e94536.2f41fe28a80f2730715b795242a01fc3dda846a85e7903adb3907dc5c5a498bf.lock


12/06/2020 04:59:27 - INFO - filelock -   Lock 139763438435184 released on /root/.cache/torch/transformers/a89d1c4637c1ea5ecd460c2a7c06a03acc9a961fc8c59aa2dd76d8a7f1e94536.2f41fe28a80f2730715b795242a01fc3dda846a85e7903adb3907dc5c5a498bf.lock


12/06/2020 04:59:52 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/06/2020 05:00:07 - INFO - farm.modeling.prediction_head -   Resizing input dimensions of MultiLabelTextClassificationHead (text_classification) from [768, 4] to [1024, 4] to match language model
12/06/2020 05:00:07 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 2e-06}'
12/06/2020 05:00:07 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
12/06/2020 05:00:07 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 840.0, 'num_training_steps': 8400}'


In [ ]:
model = trainer.train()
# f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
# f.write("\n Completed "+str(model_name)+"\n")
# f.close()
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
#[0 0 1 1 1] [0 0 1 0 0] non-hostile at end  MAY BE WROING  

12/06/2020 05:00:07 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.51it/s]
12/06/2020 05:01:55 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

Defamation F1-score:      0.33995584988962474
Fake F1-score:            0.5970149253731344
Hate F1-score:            0.45267489711934156
Offensive F1-score:       0.43006263048016696
Fine Grained F1-score:    0.47953261805286085


Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.57it/s]
12/06/2020 05:04:14 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 200 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/06/2020 05:04:14 - INFO - farm.eval -   
 _________ text_classification _________
12/06/2020 05:04:14 - INFO - farm.eval -   loss: 0.6013010613461758
12/06/2020 05:04:14 - INFO - farm.eval -   task_name: text_classification
12/06/2020 05:04:14 - INFO - farm.eval -   f1_weighted: 0.48626061068636356
12/06/2020 05:04:14 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.2926    1.0000    0.4527       110
        fake     0.4255    1.0000    0.5970       160
  defa

Defamation F1-score:      0.34267912772585674
Fake F1-score:            0.5970149253731344
Hate F1-score:            0.45267489711934156
Offensive F1-score:       0.4574209245742092
Fine Grained F1-score:    0.48626061068636356


Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.53it/s]
12/06/2020 05:06:32 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 300 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/06/2020 05:06:32 - INFO - farm.eval -   
 _________ text_classification _________
12/06/2020 05:06:32 - INFO - farm.eval -   loss: 0.5830800064066624
12/06/2020 05:06:32 - INFO - farm.eval -   task_name: text_classification
12/06/2020 05:06:32 - INFO - farm.eval -   f1_weighted: 0.5014033600586688
12/06/2020 05:06:32 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.2949    1.0000    0.4555       110
        fake     0.4290    0.9812    0.5970       160
  defam

...

Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.58it/s]
12/06/2020 05:31:25 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 1500 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/06/2020 05:31:25 - INFO - farm.eval -   
 _________ text_classification _________
12/06/2020 05:31:25 - INFO - farm.eval -   loss: 0.534303330360575
12/06/2020 05:31:25 - INFO - farm.eval -   task_name: text_classification
12/06/2020 05:31:25 - INFO - farm.eval -   f1_weighted: 0.6086037718893812
12/06/2020 05:31:25 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4143    0.7909    0.5437       110
        fake     0.8101    0.8000    0.8050       160
  defam

Defamation F1-score:      0.3488372093023256
Fake F1-score:            0.8050314465408804
Hate F1-score:            0.54375
Offensive F1-score:       0.5669291338582677
Fine Grained F1-score:    0.6086037718893812


Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.57it/s]
12/06/2020 05:33:14 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 1600 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/06/2020 05:33:14 - INFO - farm.eval -   
 _________ text_classification _________
12/06/2020 05:33:14 - INFO - farm.eval -   loss: 0.5613616508372287
12/06/2020 05:33:14 - INFO - farm.eval -   task_name: text_classification
12/06/2020 05:33:14 - INFO - farm.eval -   f1_weighted: 0.6127428552350034
12/06/2020 05:33:14 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4286    0.7909    0.5559       110
        fake     0.8377    0.8063    0.8217       160
  defa

Defamation F1-score:      0.31788079470198677
Fake F1-score:            0.821656050955414
Hate F1-score:            0.5559105431309904
Offensive F1-score:       0.5693430656934306
Fine Grained F1-score:    0.6127428552350034


12/06/2020 05:34:33 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
12/06/2020 05:34:33 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
12/06/2020 05:34:33 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [1024, 4]
12/06/2020 05:34:33 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [1.0233291387557983, 0.7087336182594299, 1.4362832307815552, 1.092193841934204]
12/06/2020 05:34:33 - INFO - farm.modeling.prediction_head -   Loading prediction head from /content/drive/My Drive/constraint2021/data/siva/saved_models/final_hierarchial_classification_model_xlmr_large_finetuned/prediction_head_0.bin


In [ ]:
w/o non-hostile 2678 sampples multi-class
11/30/2020 15:42:56 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:42:57 - INFO - farm.eval -   loss: 0.4923159746413535
11/30/2020 15:42:57 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:42:57 - INFO - farm.eval -   f1_weighted: 0.6127256099595509
11/30/2020 15:42:57 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4112    0.7364    0.5277       110
        fake     0.9098    0.6937    0.7872       160
  defamation     0.3379    0.6364    0.4414        77
   offensive     0.4171    0.8544    0.5605       103

   micro avg     0.4874    0.7311    0.5849       450
   macro avg     0.5190    0.7302    0.5792       450
weighted avg     0.5773    0.7311    0.6127       450
 samples avg     0.5381    0.7553    0.5947       450

Defamation F1-score:      0.44144144144144143
Fake F1-score:            0.7872340425531915
Hate F1-score:            0.5276872964169381
Offensive F1-score:       0.5605095541401274
Fine Grained F1-score:    0.6127256099595509

w/o non-hostile & threshold=0.25

***** EVALUATION | TEST SET | AFTER 2800 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/01/2020 09:49:05 - INFO - farm.eval -   
 _________ text_classification _________
12/01/2020 09:49:05 - INFO - farm.eval -   loss: 0.6275129280191787
12/01/2020 09:49:05 - INFO - farm.eval -   task_name: text_classification
12/01/2020 09:49:05 - INFO - farm.eval -   f1_weighted: 0.6168849658623322
12/01/2020 09:49:05 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4620    0.6636    0.5448       110
        fake     0.8472    0.7625    0.8026       160
  defamation     0.3365    0.4545    0.3867        77
   offensive     0.5074    0.6699    0.5774       103

   micro avg     0.5517    0.6644    0.6028       450
   macro avg     0.5383    0.6376    0.5779       450
weighted avg     0.5879    0.6644    0.6169       450
 samples avg     0.6042    0.6957    0.6250       450

Defamation F1-score:      0.3867403314917127
Fake F1-score:            0.8026315789473685
Hate F1-score:            0.5447761194029851
Offensive F1-score:       0.5774058577405857
Fine Grained F1-score:    0.6168849658623322

## Binary classification

In [ ]:

LABEL_LIST = ["host", "not"]
EMBEDS_DROPOUT_PROB = 0.2
LEARNING_RATE = 2e-6    #change to 5
N_EPOCHS = 50
N_GPU = 5
evaluate_every = 100
patience=7
from farm.evaluation.metrics import register_metrics
from sklearn.metrics import f1_score

def f1_weighted(preds, labels):
    f1_weighted = f1_score(y_true=labels, y_pred=preds, average="weighted")
    return {"f1_weighted": f1_weighted}

register_metrics(name="f1_weighted", implementation=f1_weighted)
#  Proportion clipped:      0.2024432809773124 with 60 seq len , 0.18 with 70  , 0.02 with 90 


In [ ]:
# for agumented farm_cleaned_augmented_t1_consthinditrain (final version )
model_name = "xlm-roberta-large"       #"bert-base-german-cased"#"xlm-roberta-large"#"bert-large-cased"#"xlm-roberta-large"#"bert-large-uncased"#"xlm-roberta-base"#"bert-large-cased"
for i in range(1):   
  tokenizer = Tokenizer.load(
    pretrained_model_name_or_path= model_name,  #"xlm-roberta-base",
    do_lower_case=False)

  random_state = 42
  processor = TextClassificationProcessor(tokenizer=tokenizer,   
                                        max_seq_len=90,   #change ?
                                        data_dir="/content/drive/My Drive/constraint2021/data/siva/",
                                        train_filename="farm_cleaned_t1_consthinditrain.tsv",       #does it accept xlsx ? or convert to cdv ?
                                        dev_filename='farm_cleaned_t1_consthindivalid.tsv',
                                        test_filename=None,
                                        # test_filename="farm_cleaned_t1_consthindivalid.tsv",   # HAVING NONE HERE IS VERY IMPORTANT .. o/w it will load from test.tsv and in your case, it has wrong columns 
                                        # dev_split=0.075,      #for 0.1, 1400/5400 goes away 
                                        label_list=LABEL_LIST,
                                        metric="f1_weighted",
                                        #  multilabel=True,
                                        label_column_name="bin_label")
  BATCH_SIZE = 16#32

  data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

  print("Seed set is {}".format(random_state))
  torch.manual_seed(random_state)
  prediction_head =TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

  MODEL_NAME_OR_PATH =  model_name   #"xlm-roberta-base"     #"bert-base-multilingual-uncased"
  language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

  # language_model = AdaptiveModel.convert_from_transformers('xlnet-base-cased', device=torch.device('cuda:0'), task_type="text_classification")
  model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=torch.device('cuda:0'))



  model, optimizer, lr_schedule = initialize_optimizer(
      model=model,
      device=torch.device('cuda:0'),
      learning_rate=LEARNING_RATE,
      n_batches=len(data_silo.loaders["train"]),
      n_epochs=N_EPOCHS)



  # DEfine Early Stopping:
  earlystopping = EarlyStopping(metric='f1_weighted',
                                save_dir='/content/drive/My Drive/constraint2021/data/siva/saved_models/final_non_augmented_bin_classification_model_xlmr_large_finetuned',
                                patience=patience,
                                mode='max'  # telling it to maximize the f1-score/metric. I f loss was here, min will be the mode
                                )
  # You can also load from early stopping best model as follows : model = Inferencer.load(earlystopping.save_dir) 
  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      data_silo=data_silo,
      epochs=N_EPOCHS,
      n_gpu=N_GPU,
      evaluate_every = evaluate_every,
      early_stopping=earlystopping,
      lr_schedule=lr_schedule,
      device=torch.device('cuda:0'),
      disable_tqdm=True
  )

  model = trainer.train()
  # f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
  # f.write("\n Completed "+str(model_name)+"\n")
  # f.close()


    


12/22/2020 11:01:57 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'XLMRobertaTokenizer'
12/22/2020 11:01:57 - INFO - filelock -   Lock 139640121571200 acquired on /root/.cache/torch/transformers/f7e58cf8eef122765ff522a4c7c0805d2fe8871ec58dcb13d0c2764ea3e4a0f3.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed.lock


12/22/2020 11:01:58 - INFO - filelock -   Lock 139640121571200 released on /root/.cache/torch/transformers/f7e58cf8eef122765ff522a4c7c0805d2fe8871ec58dcb13d0c2764ea3e4a0f3.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed.lock


12/22/2020 11:01:59 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
12/22/2020 11:01:59 - INFO - farm.data_handler.data_silo -   Loading train set from: /content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthinditrain.tsv 
12/22/2020 11:02:02 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 5728 dictionaries to pytorch datasets (chunksize = 1146)...
12/22/2020 11:02:02 - INFO - farm.data_handler.data_silo -    0 
12/22/2020 11:02:02 - INFO - farm.data_handler.data_silo -   /w\
12/22/2020 11:02:02 - INFO - farm.data_handler.data_silo -   / \
12/22/2020 11:02:02 - INFO - farm.data_handler.data_silo -   
Preprocessing Dataset /content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthinditrain.tsv:   0%|          | 0/5728 [00:00<?, ? Dicts/s]12/22/2020 11:02:04 

Seed set is 42


12/22/2020 11:02:15 - INFO - filelock -   Lock 139640106995048 acquired on /root/.cache/torch/transformers/5ac6d3984e5ca7c5227e4821c65d341900125db538c5f09a1ead14f380def4a7.aa59609b4f56f82fa7699f0d47997566ccc4cf07e484f3a7bc883bd7c5a34488.lock


12/22/2020 11:02:15 - INFO - filelock -   Lock 139640106995048 released on /root/.cache/torch/transformers/5ac6d3984e5ca7c5227e4821c65d341900125db538c5f09a1ead14f380def4a7.aa59609b4f56f82fa7699f0d47997566ccc4cf07e484f3a7bc883bd7c5a34488.lock


12/22/2020 11:02:16 - INFO - filelock -   Lock 139640106838168 acquired on /root/.cache/torch/transformers/a89d1c4637c1ea5ecd460c2a7c06a03acc9a961fc8c59aa2dd76d8a7f1e94536.2f41fe28a80f2730715b795242a01fc3dda846a85e7903adb3907dc5c5a498bf.lock


12/22/2020 11:03:19 - INFO - filelock -   Lock 139640106838168 released on /root/.cache/torch/transformers/a89d1c4637c1ea5ecd460c2a7c06a03acc9a961fc8c59aa2dd76d8a7f1e94536.2f41fe28a80f2730715b795242a01fc3dda846a85e7903adb3907dc5c5a498bf.lock


12/22/2020 11:03:41 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/22/2020 11:03:56 - INFO - farm.modeling.prediction_head -   Resizing input dimensions of TextClassificationHead (text_classification) from [768, 2] to [1024, 2] to match language model
12/22/2020 11:03:56 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 2e-06}'
12/22/2020 11:03:56 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
12/22/2020 11:03:56 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 1790.0, 'num_training_steps': 17900}'
12/22/2020 11:03:56 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _          

In [ ]:
model

AdaptiveModel(
  (language_model): XLMRoberta(
    (model): XLMRobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(250002, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=1024,

In [ ]:
               precision    recall  f1-score   support

        host     0.9173    0.9149    0.9161       376
         not     0.9266    0.9287    0.9277       435

    accuracy                         0.9223       811
   macro avg     0.9220    0.9218    0.9219       811
weighted avg     0.9223    0.9223    0.9223       811
WITH DATA AUGUM..


## Multi-class classification

In [ ]:

LABEL_LIST = ["hate", "fake","defamation","offensive","non-hostile"]
EMBEDS_DROPOUT_PROB = 0.2
LEARNING_RATE = 2e-6    #change to 5
N_EPOCHS = 50
N_GPU = 5
evaluate_every = 100
patience=7
from farm.evaluation.metrics import register_metrics
from sklearn.metrics import f1_score

# def f1_weighted(preds, labels):
#     f1_weighted = f1_score(y_true=labels, y_pred=preds, average="weighted",zero_division=1)
#     return {"f1_weighted": f1_weighted}

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score



def preprocess(df):
    
    df = df.dropna()
    
    df.insert(len(df.columns)-1,'Hostile', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Defamation', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Fake', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Hate', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Offensive', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Non-Hostile', np.zeros(len(df),dtype=int))    
    
    for i in range(len(df)):
        text = df['Labels Set'][i]
        text = text.lower()
        text = text.replace('\n',"")
        text = text.replace('"',"")
        text = text.replace(" ","")
        text = text.split(',')


        for word in text:
            # print(word)
            if word == 'defamation':
                df.at[i,'Hostile']    = 1
                df.at[i,'Defamation'] = 1
    
            if word == 'fake':
                df.at[i,'Hostile']    = 1
                df.at[i,'Fake'] = 1
    
            if word == 'hate':
                df.at[i,'Hostile']    = 1
                df.at[i,'Hate'] = 1
    
            if word == 'offensive':
                df.at[i,'Hostile']    = 1
                df.at[i,'Offensive'] = 1
    
            if word == 'non-hostile' and df['Hostile'][i]==0:
                df.at[i,'Hostile']    = 0
                df.at[i,'Non-Hostile'] = 1

    return df 
  
    



def get_scores(y_true, y_pred):
    
    hostility_true = y_true['Hostile']
    hostility_pred = y_pred['Hostile']
    print(hostility_true,hostility_pred)
    hostility_f1 = f1_score(y_true=hostility_true, y_pred=hostility_pred, average='weighted')
    
    fine_true = y_true[['Defamation','Fake','Hate','Offensive']]
    fine_pred = y_pred[['Defamation','Fake','Hate','Offensive']]
    
    fine_f1          = f1_score(y_true=fine_true, y_pred=fine_pred, average=None)
    defame_f1        = fine_f1[0]
    fake_f1          = fine_f1[1]
    hate_f1          = fine_f1[2]
    offensive_f1     = fine_f1[3]
    weighted_fine_f1 = f1_score(y_true=fine_true, y_pred=fine_pred, average='weighted')

    return [hostility_f1, defame_f1, fake_f1, hate_f1, offensive_f1, weighted_fine_f1]
i=0
def f1_weighted(preds, labels):
    i=0
    if(i==0):
        # print(preds,labels)
        i+=1
    
    # df_pred = pd.DataFrame(index=None)
    # df_pred = df_pred.assign(Unique_ID=[i for i in range(1,len(preds)+1)])
    # df_pred = df_pred.assign(Labels_Set = preds )
    # df_pred=df_pred.rename(columns={'Unique_ID':'Unique ID','Labels_Set':'Labels Set' })

    # df_true = pd.DataFrame(index=None)
    # df_true = df_true.assign(Unique_ID=[i for i in range(1,len(labels)+1)])
    # df_true = df_true.assign(Labels_Set = labels)
    # df_true=df_true.rename(columns={'Unique_ID':'Unique ID','Labels_Set':'Labels Set' })
    # df_true = preprocess(df_true)
    # df_pred = preprocess(df_pred)

    # f1_weighted = get_scores(df_true,df_pred)[-1]

    labels = labels[:,:-1]
    preds = preds[:,:-1]

    fine_f1          = f1_score(y_true=labels, y_pred=preds, average=None)
    defame_f1        = fine_f1[0]
    fake_f1          = fine_f1[1]
    hate_f1          = fine_f1[2]
    offensive_f1     = fine_f1[3]
    print("coarse F1-score:     ", fine_f1)
    print("Defamation F1-score:     ", defame_f1)
    print("Fake F1-score:           ", fake_f1)
    print("Hate F1-score:           ", hate_f1)
    print("Offensive F1-score:      ", offensive_f1)
    weighted_fine_f1 = f1_score(y_true=labels, y_pred=preds, average='weighted')
    print("Fine Grained F1-score:   ",weighted_fine_f1)
    
    return {"f1_weighted": weighted_fine_f1}

register_metrics(name="f1_weighted", implementation=f1_weighted)



In [ ]:

model_name = "xlm-roberta-large"       #"bert-base-german-cased"#"xlm-roberta-large"#"bert-large-cased"#"xlm-roberta-large"#"bert-large-uncased"#"xlm-roberta-base"#"bert-large-cased"
for i in range(1):   
  tokenizer = Tokenizer.load(
    pretrained_model_name_or_path= model_name,  #"xlm-roberta-base",
    do_lower_case=False)

  random_state = 2
  processor = TextClassificationProcessor(tokenizer=tokenizer,#delimiter=',',   
                                        max_seq_len=90,   #change ?
                                        data_dir="/content/drive/My Drive/constraint2021/data/siva/",
                                        train_filename="farm_uncleaned_train.tsv",       #does it accept xlsx ? or convert to cdv ?
                                        dev_filename='farm_uncleaned_valid.tsv',
                                        test_filename="farm_uncleaned_test.tsv",   # HAVING NONE HERE IS VERY IMPORTANT .. o/w it will load from test.tsv and in your case, it has wrong columns 
                                        # dev_split=0.075,      #for 0.1, 1400/5400 goes away 
                                        label_list=LABEL_LIST,
                                        multilabel=True,
                                        metric="f1_weighted",
                                        label_column_name="multi_label")
  BATCH_SIZE = 16#32#3#2

  data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

  print("Seed set is {}".format(random_state))
  torch.manual_seed(random_state)
  prediction_head = MultiLabelTextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"),pred_threshold=0.4)

  MODEL_NAME_OR_PATH =  model_name   #"xlm-roberta-base"     #"bert-base-multilingual-uncased"
  language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

  # language_model = AdaptiveModel.convert_from_transformers('xlnet-base-cased', device=torch.device('cuda:0'), task_type="text_classification")
  model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=torch.device('cuda:0'))



  model, optimizer, lr_schedule = initialize_optimizer(
      model=model,
      device=torch.device('cuda:0'),
      learning_rate=LEARNING_RATE,
      n_batches=len(data_silo.loaders["train"]),
      n_epochs=N_EPOCHS)



  # DEfine Early Stopping:
  earlystopping = EarlyStopping(metric='f1_weighted',
                                save_dir='/content/drive/My Drive/constraint2021/data/siva/saved_models/final_multi_label_uncleaned_all_model',
                                patience=patience,
                                mode='max'  #  telling it to maximize the f1-score/metric. I f loss was here, min will be the mode
                                )
  # You can also load from early stopping best model as follows : model = Inferencer.load(earlystopping.save_dir)
  #  
  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      data_silo=data_silo,
      epochs=N_EPOCHS,
      n_gpu=N_GPU,
      evaluate_every = evaluate_every,
      early_stopping=earlystopping,
      lr_schedule=lr_schedule,
      device=torch.device('cuda:0'),
      disable_tqdm=True
  )




    


In [ ]:
model = trainer.train()
# f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
# f.write("\n Completed "+str(model_name)+"\n")
# f.close()
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
#[0 0 1 1 1] [0 0 1 0 0] non-hostile at end  MAY BE WROING  

12/23/2020 10:53:34 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 84/84 [00:21<00:00,  3.83it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division`

coarse F1-score:      [0.06378132 0.08836524 0.05523256 0.08309456]
Defamation F1-score:      0.06378132118451026
Fake F1-score:            0.08836524300441827
Hate F1-score:            0.055232558139534885
Offensive F1-score:       0.0830945558739255
Fine Grained F1-score:    0.07519890251431827


Evaluating: 100%|██████████| 84/84 [00:22<00:00,  3.67it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
12/23/2020 10:58:06 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 200 BATCHES *****
***************************************************
\\|//     

coarse F1-score:      [0.03076923 0.         0.14379085 0.10795455]
Defamation F1-score:      0.030769230769230767
Fake F1-score:            0.0
Hate F1-score:            0.14379084967320263
Offensive F1-score:       0.10795454545454546
Fine Grained F1-score:    0.06507095561090294


Evaluating: 100%|██████████| 84/84 [00:23<00:00,  3.51it/s]
12/23/2020 11:00:11 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 300 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/23/2020 11:00:11 - INFO - farm.eval -   
 _________ text_classification _________
12/23/2020 11:00:11 - INFO - farm.eval -   loss: 0.3164232304634357
12/23/2020 11:00:11 - INFO - farm.eval -   task_name: text_classification
12/23/2020 11:00:11 - INFO - farm.eval -   f1_weighted: 0.03952838546991072
12/23/2020 11:00:11 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.0000    0.0000    0.0000        44
        fake     0.0000    0.0000    0.0000        61
  defa

coarse F1-score:      [0.        0.        0.        0.1369863]
Defamation F1-score:      0.0
Fake F1-score:            0.0
Hate F1-score:            0.0
Offensive F1-score:       0.136986301369863
Fine Grained F1-score:    0.03952838546991072


Evaluating: 100%|██████████| 84/84 [00:23<00:00,  3.54it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
12/23/2020 11:02:17 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 400 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/23/2020 11:02:17 - INFO - farm.eval -   
 _________ text_classification _________
12/23/2020 11:02:17 - INFO - farm.eval -   loss: 0.22253264520200378
12/23/2020 11:02:17 - INFO - farm.eval -   task_name: text_classific

coarse F1-score:      [0. 0. 0. 0.]
Defamation F1-score:      0.0
Fake F1-score:            0.0
Hate F1-score:            0.0
Offensive F1-score:       0.0
Fine Grained F1-score:    0.0


KeyboardInterrupt: ignored

###IT's results

In [ ]:
THRESHOLD 0.4 

11/30/2020 06:34:44 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 06:34:44 - INFO - farm.eval -   loss: 0.35257818034486027
11/30/2020 06:34:44 - INFO - farm.eval -   task_name: text_classification
11/30/2020 06:34:44 - INFO - farm.eval -   f1_weighted: 0.7302623095977058
11/30/2020 06:34:44 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4530    0.7455    0.5636       110
        fake     0.6194    0.6000    0.6095       160
 non-hostile     0.9284    0.8943    0.9110       435
  defamation     0.3529    0.6234    0.4507        77
   offensive     0.4921    0.6019    0.5415       103

   micro avg     0.6657    0.7650    0.7119       885
   macro avg     0.5692    0.6930    0.6153       885
weighted avg     0.7126    0.7650    0.7303       885
 samples avg     0.7187    0.7803    0.7340       885

 

THRESHOLD 0.4 ON THEIR METRIC 




 ***** EVALUATION | TEST SET | AFTER 2600 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 12:04:45 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 12:04:45 - INFO - farm.eval -   loss: 0.3053237624585849
11/30/2020 12:04:45 - INFO - farm.eval -   task_name: text_classification
11/30/2020 12:04:45 - INFO - farm.eval -   f1_weighted: 0.5534680346052169
11/30/2020 12:04:45 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4055    0.8000    0.5382       110
        fake     0.6460    0.6500    0.6480       160
  defamation     0.2814    0.7273    0.4058        77
   offensive     0.4474    0.6602    0.5333       103
 non-hostile     0.9311    0.9011    0.9159       435

   micro avg     0.6157    0.8000    0.6958       885
   macro avg     0.5423    0.7477    0.6082       885
weighted avg     0.7014    0.8000    0.7316       885
 samples avg     0.6965    0.8088    0.7268       885

Defamation F1-score:      0.5382262996941897
Fake F1-score:            0.64797507788162
Hate F1-score:            0.40579710144927533
Offensive F1-score:       0.5333333333333333
Fine Grained F1-score:    0.5534680346052169




w/o non-hostile 2678 sampples multi-class
11/30/2020 15:42:56 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:42:57 - INFO - farm.eval -   loss: 0.4923159746413535
11/30/2020 15:42:57 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:42:57 - INFO - farm.eval -   f1_weighted: 0.6127256099595509
11/30/2020 15:42:57 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4112    0.7364    0.5277       110
        fake     0.9098    0.6937    0.7872       160
  defamation     0.3379    0.6364    0.4414        77
   offensive     0.4171    0.8544    0.5605       103

   micro avg     0.4874    0.7311    0.5849       450
   macro avg     0.5190    0.7302    0.5792       450
weighted avg     0.5773    0.7311    0.6127       450
 samples avg     0.5381    0.7553    0.5947       450

Defamation F1-score:      0.44144144144144143
Fake F1-score:            0.7872340425531915
Hate F1-score:            0.5276872964169381
Offensive F1-score:       0.5605095541401274
Fine Grained F1-score:    0.6127256099595509


w/o non-hostile augmented 

***** EVALUATION | TEST SET | AFTER 6900 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/01/2020 06:40:33 - INFO - farm.eval -   
 _________ text_classification _________
12/01/2020 06:40:33 - INFO - farm.eval -   loss: 1.0844329418020044
12/01/2020 06:40:33 - INFO - farm.eval -   task_name: text_classification
12/01/2020 06:40:33 - INFO - farm.eval -   f1_weighted: 0.5780401625053231
12/01/2020 06:40:33 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.4889    0.6000    0.5388       110
        fake     0.8682    0.7000    0.7751       160
  defamation     0.3472    0.3247    0.3356        77
   offensive     0.4951    0.4951    0.4951       103

   micro avg     0.5786    0.5644    0.5714       450
   macro avg     0.5499    0.5300    0.5361       450
weighted avg     0.6010    0.5644    0.5780       450
 samples avg     0.5811    0.5898    0.5690       450

Defamation F1-score:      0.33557046979865773
Fake F1-score:            0.7750865051903114
Hate F1-score:            0.5387755102040817
Offensive F1-score:       0.49514563106796117
Fine Grained F1-score:    0.5780401625053231


In [ ]:
model = trainer.train()
# f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
# f.write("\n Completed "+str(model_name)+"\n")
# f.close()
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
#[0 0 1 1 1] [0 0 1 0 0] non-hostile at end  MAY BE WROING  

12/01/2020 04:13:51 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 63/63 [00:17<00:00,  3.63it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_d

Defamation F1-score:      0.0
Fake F1-score:            0.5813788201847904
Hate F1-score:            0.4844343204252088
Offensive F1-score:       0.447900466562986
Fine Grained F1-score:    0.4269564455703201


Evaluating: 100%|██████████| 63/63 [00:17<00:00,  3.65it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
12/01/2020 04:18:11 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 200 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/01/2020 04:18:11 - INFO - farm.eval -   
 _________ text_classification _________
12/01/2020 04:18:11 - INFO - farm.eval -   loss: 0.6137540569764101
12/01/2020 04:18:11 - INFO - farm.eval -   task_name: text_cl

Defamation F1-score:      0.0
Fake F1-score:            0.27320490367775824
Hate F1-score:            0.0
Offensive F1-score:       0.0
Fine Grained F1-score:    0.09151581130565366


Evaluating: 100%|██████████| 63/63 [00:17<00:00,  3.68it/s]
12/01/2020 04:20:08 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 300 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/01/2020 04:20:08 - INFO - farm.eval -   
 _________ text_classification _________
12/01/2020 04:20:08 - INFO - farm.eval -   loss: 0.6052547862868988
12/01/2020 04:20:08 - INFO - farm.eval -   task_name: text_classification
12/01/2020 04:20:08 - INFO - farm.eval -   f1_weighted: 0.11157301076022619
12/01/2020 04:20:08 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.0000    0.0000    0.0000       319
        fake     0.6852    0.1809    0.2863       409
  defa

...

Defamation F1-score:      0.577889447236181
Fake F1-score:            0.8599508599508598
Hate F1-score:            0.7034700315457414
Offensive F1-score:       0.6654135338345865
Fine Grained F1-score:    0.7258257790424006


Evaluating: 100%|██████████| 63/63 [00:17<00:00,  3.67it/s]
12/01/2020 06:39:30 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 6900 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

12/01/2020 06:39:30 - INFO - farm.eval -   
 _________ text_classification _________
12/01/2020 06:39:30 - INFO - farm.eval -   loss: 0.6957476283600909
12/01/2020 06:39:30 - INFO - farm.eval -   task_name: text_classification
12/01/2020 06:39:30 - INFO - farm.eval -   f1_weighted: 0.7161957369079441
12/01/2020 06:39:30 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.6564    0.7367    0.6942       319
        fake     0.8668    0.8435    0.8550       409
  defa

Defamation F1-score:      0.5643835616438356
Fake F1-score:            0.8550185873605949
Hate F1-score:            0.6942392909896602
Offensive F1-score:       0.6514285714285715
Fine Grained F1-score:    0.7161957369079441


12/01/2020 06:40:26 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
12/01/2020 06:40:26 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
12/01/2020 06:40:26 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [1024, 4]
12/01/2020 06:40:26 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [1.036234974861145, 0.7124115824699402, 1.4131579399108887, 1.0826612710952759]
12/01/2020 06:40:26 - INFO - farm.modeling.prediction_head -   Loading prediction head from /content/drive/My Drive/constraint2021/data/siva/saved_models/hierar_augme_wo_non_hostile_threshoild_40percent42-xlm-roberta-large-2e-5-50epochs_patience7/prediction_head_0.bin
Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.79it/s]
12/01/2020 06:40:33 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Defamation F1-score:      0.33557046979865773
Fake F1-score:            0.7750865051903114
Hate F1-score:            0.5387755102040817
Offensive F1-score:       0.49514563106796117
Fine Grained F1-score:    0.5780401625053231


In [ ]:
model = trainer.train()
# f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
# f.write("\n Completed "+str(model_name)+"\n")
# f.close()
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
#[0 0 1 1 1] [0 0 1 0 0] non-hostile at end  MAY BE WROING  

11/30/2020 15:12:01 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

11/30/2020 15:12:01 - INFO - farm.train -   Skipping 0 out of 100 steps ...
11/30/2020 15:12:01 - INFO - farm.train -   Finished skipping 100 steps ...
Evaluating: 100%|██████████| 34/34 [00:08<00:00,  4.05it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_cla

Defamation F1-score:      0.0
Fake F1-score:            0.6075619295958279
Hate F1-score:            0.4693877551020408
Offensive F1-score:       0.42253521126760557
Fine Grained F1-score:    0.42893135602850735


Evaluating: 100%|██████████| 34/34 [00:08<00:00,  3.85it/s]
11/30/2020 15:14:25 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 200 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 15:14:25 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:14:25 - INFO - farm.eval -   loss: 0.5971664837683631
11/30/2020 15:14:25 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:14:25 - INFO - farm.eval -   f1_weighted: 0.16755422327346325
11/30/2020 15:14:25 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.0000    0.0000    0.0000       165
        fake     0.7282    0.3205    0.4451       234
  defa

Defamation F1-score:      0.017543859649122806
Fake F1-score:            0.44510385756676557
Hate F1-score:            0.0
Offensive F1-score:       0.014598540145985401
Fine Grained F1-score:    0.16755422327346325


Evaluating: 100%|██████████| 34/34 [00:09<00:00,  3.75it/s]
11/30/2020 15:16:13 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 300 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 15:16:13 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:16:13 - INFO - farm.eval -   loss: 0.5737895536958502
11/30/2020 15:16:13 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:16:13 - INFO - farm.eval -   f1_weighted: 0.3053149509579362
11/30/2020 15:16:13 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.0000    0.0000    0.0000       165
        fake     0.7385    0.6880    0.7124       234
  defam

...

Defamation F1-score:      0.23204419889502764
Fake F1-score:            0.7903930131004367
Hate F1-score:            0.5768194070080863
Offensive F1-score:       0.5401929260450161
Fine Grained F1-score:    0.5873025090159095


Evaluating: 100%|██████████| 34/34 [00:09<00:00,  3.69it/s]
11/30/2020 15:41:49 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 1600 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 15:41:49 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:41:49 - INFO - farm.eval -   loss: 0.5821610416812397
11/30/2020 15:41:49 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:41:49 - INFO - farm.eval -   f1_weighted: 0.587347612544763
11/30/2020 15:41:49 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.5137    0.5697    0.5402       165
        fake     0.7925    0.8162    0.8042       234
  defam

Defamation F1-score:      0.24175824175824176
Fake F1-score:            0.8042105263157894
Hate F1-score:            0.5402298850574713
Offensive F1-score:       0.5531914893617021
Fine Grained F1-score:    0.587347612544763


11/30/2020 15:42:49 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
11/30/2020 15:42:49 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
11/30/2020 15:42:49 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [1024, 4]
11/30/2020 15:42:49 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [1.0354299545288086, 0.7137760519981384, 1.432268738746643, 1.0694900751113892]
11/30/2020 15:42:49 - INFO - farm.modeling.prediction_head -   Loading prediction head from /content/drive/My Drive/constraint2021/data/siva/saved_models/hierar_wo_non_hostile_threshoild_40percent42-xlm-roberta-large-2e-5-50epochs_patience7/prediction_head_0.bin
Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.88it/s]
11/30/2020 15:42:56 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Defamation F1-score:      0.44144144144144143
Fake F1-score:            0.7872340425531915
Hate F1-score:            0.5276872964169381
Offensive F1-score:       0.5605095541401274
Fine Grained F1-score:    0.6127256099595509


In [ ]:
model = trainer.train()
# f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
# f.write("\n Completed "+str(model_name)+"\n")
# f.close()
#/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
#[0 0 1 1 1] [0 0 1 0 0] non-hostile at end  MAY BE WROING  

11/30/2020 15:12:01 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

11/30/2020 15:12:01 - INFO - farm.train -   Skipping 0 out of 100 steps ...
11/30/2020 15:12:01 - INFO - farm.train -   Finished skipping 100 steps ...
Evaluating: 100%|██████████| 34/34 [00:08<00:00,  4.05it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_cla

Defamation F1-score:      0.0
Fake F1-score:            0.6075619295958279
Hate F1-score:            0.4693877551020408
Offensive F1-score:       0.42253521126760557
Fine Grained F1-score:    0.42893135602850735


Evaluating: 100%|██████████| 34/34 [00:08<00:00,  3.85it/s]
11/30/2020 15:14:25 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 200 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 15:14:25 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:14:25 - INFO - farm.eval -   loss: 0.5971664837683631
11/30/2020 15:14:25 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:14:25 - INFO - farm.eval -   f1_weighted: 0.16755422327346325
11/30/2020 15:14:25 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.0000    0.0000    0.0000       165
        fake     0.7282    0.3205    0.4451       234
  defa

Defamation F1-score:      0.017543859649122806
Fake F1-score:            0.44510385756676557
Hate F1-score:            0.0
Offensive F1-score:       0.014598540145985401
Fine Grained F1-score:    0.16755422327346325


Evaluating: 100%|██████████| 34/34 [00:09<00:00,  3.75it/s]
11/30/2020 15:16:13 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 300 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 15:16:13 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:16:13 - INFO - farm.eval -   loss: 0.5737895536958502
11/30/2020 15:16:13 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:16:13 - INFO - farm.eval -   f1_weighted: 0.3053149509579362
11/30/2020 15:16:13 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.0000    0.0000    0.0000       165
        fake     0.7385    0.6880    0.7124       234
  defam

...

Defamation F1-score:      0.23204419889502764
Fake F1-score:            0.7903930131004367
Hate F1-score:            0.5768194070080863
Offensive F1-score:       0.5401929260450161
Fine Grained F1-score:    0.5873025090159095


Evaluating: 100%|██████████| 34/34 [00:09<00:00,  3.69it/s]
11/30/2020 15:41:49 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 1600 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

11/30/2020 15:41:49 - INFO - farm.eval -   
 _________ text_classification _________
11/30/2020 15:41:49 - INFO - farm.eval -   loss: 0.5821610416812397
11/30/2020 15:41:49 - INFO - farm.eval -   task_name: text_classification
11/30/2020 15:41:49 - INFO - farm.eval -   f1_weighted: 0.587347612544763
11/30/2020 15:41:49 - INFO - farm.eval -   report: 
               precision    recall  f1-score   support

        hate     0.5137    0.5697    0.5402       165
        fake     0.7925    0.8162    0.8042       234
  defam

Defamation F1-score:      0.24175824175824176
Fake F1-score:            0.8042105263157894
Hate F1-score:            0.5402298850574713
Offensive F1-score:       0.5531914893617021
Fine Grained F1-score:    0.587347612544763


11/30/2020 15:42:49 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
11/30/2020 15:42:49 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
11/30/2020 15:42:49 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [1024, 4]
11/30/2020 15:42:49 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [1.0354299545288086, 0.7137760519981384, 1.432268738746643, 1.0694900751113892]
11/30/2020 15:42:49 - INFO - farm.modeling.prediction_head -   Loading prediction head from /content/drive/My Drive/constraint2021/data/siva/saved_models/hierar_wo_non_hostile_threshoild_40percent42-xlm-roberta-large-2e-5-50epochs_patience7/prediction_head_0.bin
Evaluating: 100%|██████████| 24/24 [00:06<00:00,  3.88it/s]
11/30/2020 15:42:56 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Defamation F1-score:      0.44144144144144143
Fake F1-score:            0.7872340425531915
Hate F1-score:            0.5276872964169381
Offensive F1-score:       0.5605095541401274
Fine Grained F1-score:    0.6127256099595509


## Hindi- BERT

In [ ]:

LABEL_LIST = ["host", "not"]
EMBEDS_DROPOUT_PROB = 0.2
LEARNING_RATE = 2e-6    #change to 5
N_EPOCHS = 50
N_GPU = 5
evaluate_every = 100
patience=7
from farm.evaluation.metrics import register_metrics
from sklearn.metrics import f1_score

def f1_weighted(preds, labels):
    f1_weighted = f1_score(y_true=labels, y_pred=preds, average="weighted")
    return {"f1_weighted": f1_weighted}

register_metrics(name="f1_weighted", implementation=f1_weighted)
#  Proportion clipped:      0.2024432809773124 with 60 seq len , 0.18 with 70  , 0.02 with 90 


In [ ]:

model_name = "bert-base-uncased"#"xlm-roberta-large"       #"bert-base-german-cased"#"xlm-roberta-large"#"bert-large-cased"#"xlm-roberta-large"#"bert-large-uncased"#"xlm-roberta-base"#"bert-large-cased"

for i in range(1):   
  tokenizer = Tokenizer.load(
    pretrained_model_name_or_path= model_name,  #"xlm-roberta-base",
    do_lower_case=False)

  random_state = 42

  # model = AdaptiveModel.convert_from_transformers(model_name_or_path, device, task_type)
  # processor = Processor.convert_from_transformers(model_name_or_path, task_type, max_seq_len, doc_stride, tokenizer_class, tokenizer_args, use_fast)
            
            
  processor = TextClassificationProcessor(tokenizer=tokenizer,   
                                        max_seq_len=90,   #change ?
                                        data_dir="/content/drive/My Drive/constraint2021/data/siva/",
                                        train_filename="farm_cleaned_t1_consthinditrain.tsv",       #does it accept xlsx ? or convert to cdv ?
                                        # dev_filename='farm_hindi_dev_20_task1split.tsv',
                                        test_filename="farm_cleaned_t1_consthindivalid.tsv",   # HAVING NONE HERE IS VERY IMPORTANT .. o/w it will load from test.tsv and in your case, it has wrong columns 
                                        dev_split=0.075,      #for 0.1, 1400/5400 goes away 
                                        label_list=LABEL_LIST,
                                        metric="f1_weighted",
                                        #  multilabel=True,
                                        label_column_name="bin_label")
  BATCH_SIZE = 8#32

  data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

  print("Seed set is {}".format(random_state))
  torch.manual_seed(random_state)
  prediction_head =TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

  # MODEL_NAME_OR_PATH =  model_name   #"xlm-roberta-base"     #"bert-base-multilingual-uncased"
  # language_model = LanguageModel.load(MODEL_NAME_OR_PATH)


  model = AdaptiveModel.convert_from_transformers("monsoon-nlp/hindi-bert", device="cpu", task_type="text_classification",processor=processor)
  # language_model = AdaptiveModel.convert_from_transformers('xlnet-base-cased', device=torch.device('cuda:0'), task_type="text_classification")
  model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=torch.device('cuda:0'))



  model, optimizer, lr_schedule = initialize_optimizer(
      model=model,
      device=torch.device('cuda:0'),
      learning_rate=LEARNING_RATE,
      n_batches=len(data_silo.loaders["train"]),
      n_epochs=N_EPOCHS)



  # DEfine Early Stopping:
  earlystopping = EarlyStopping(metric='f1_weighted',
                                save_dir='/content/drive/My Drive/constraint2021/data/siva/saved_models/hindi-bert'+str(random_state)+'-'+str(model_name)+'-2e-5-50epochs_patience7',
                                patience=patience,
                                mode='max'  # telling it to maximize the f1-score/metric. I f loss was here, min will be the mode
                                )
  # You can also load from early stopping best model as follows : model = Inferencer.load(earlystopping.save_dir) 
  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      data_silo=data_silo,
      epochs=N_EPOCHS,
      n_gpu=N_GPU,
      evaluate_every = evaluate_every,
      early_stopping=earlystopping,
      lr_schedule=lr_schedule,
      device=torch.device('cuda:0'),
      disable_tqdm=True
  )

  model = trainer.train()
  # f = open("/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/updates/train_updates.txt", "a")
  # f.write("\n Completed "+str(model_name)+"\n")
  # f.close()


    


## Trying HinBERT with simple-transformers library

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
!pip install simpletransformers==0.48.0

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.4MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 317kB 13.5MB/s 
     |████████████████████████████████| 1.7MB 18.7MB/s 
     |████████████████████████████████| 2.9MB 34.3MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 7.4MB 38.3MB/s 
     |████████████████████████████████| 1.1MB 46.0MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
     |████████████████████████████████| 102kB 6.9MB/s 
     |████████████████████████████████| 122kB 43.1MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 163kB 46.4MB/s 
     |████████████████████████████████| 133kB 45.8MB/s 
     |████████████████████████████████| 112kB 32.9MB/s 
     |████████████████████████████████| 6.7MB 43.9MB/s 
     |████████████████████████████████| 4.4MB 42.3MB/s 

In [ ]:
from simpletransformers.classification import ClassificationModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


ImportError: ignored

In [ ]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthinditrain.tsv",sep='\t')
train = train[['text', 'bin_label']]

test = pd.read_csv("/content/drive/My Drive/constraint2021/data/siva/farm_cleaned_t1_consthindivalid.tsv",sep='\t')
test = test[['text', 'bin_label']]

* SET cuda true on gpu.. 

In [ ]:
# set use_cuda=False on CPU-only platforms
model = ClassificationModel('xlmroberta', 'xlm-roberta-base', use_cuda=False, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 5
    # 'silent': True
})
model.train_model(train)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:751: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


## Get Labels

In [ ]:
# !pip install torch==1.5.0+cu92 torchvision==0.6.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html -q

import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))
!pip install urllib3==1.25.11 -q
!pip install farm -q


Torch 1.7.0+cu101 CUDA 10.1
Device: cuda:0
     |████████████████████████████████| 133kB 16.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 215kB 13.5MB/s 
     |████████████████████████████████| 327kB 35.4MB/s 
     |████████████████████████████████| 1.1MB 58.6MB/s 
     |████████████████████████████████| 748.8MB 24kB/s 
     |████████████████████████████████| 47.7MB 67kB/s 
     |████████████████████████████████| 133kB 60.9MB/s 
     |████████████████████████████████| 2.5MB 54.3MB/s 
     |████████████████████████████████| 3.0MB 46.4MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 
     |████████████████████████████████| 1.1MB 50.2MB/s 
     |████████████████████████████████| 81kB 13.5MB/s 
     |████████████████████████████████| 61kB 11.5MB/s 
     |████████████████████████████████| 163kB 61.0MB/s 
     |████████████████████████████████| 163kB 66.9MB/s 
     |█████

In [ ]:
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer,EarlyStopping
from farm.utils import MLFlowLogger
from farm.infer import Inferencer
from farm.evaluation.metrics import register_metrics


12/23/2020 07:04:12 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
import numpy as np

def get_pred_labels(num_samples,inferenced_model,test):

  print(num_samples)
  texts = []
  sample = test['text'][:num_samples]
  res = get_res(inferenced_model,sample)

  pred_labels =  np.array(['a' for a in range(num_samples)], dtype='object')
  # temp = np.zeros(shape=(1,2))
  i=0
  for lis in res:    # in resurlt dict
    list_of_dicts = lis['predictions']
    for dic in list_of_dicts:
      # if(int(dic['label'])==0):
      #   temp[0][0] = dic['probability']
      #   temp[0][1] = 1-dic['probability']
      # else:
      #   temp[0][1] = dic['probability']
      #   temp[0][0] = 1-dic['probability']
      # # break
      pred_labels[i] = dic['label']
      i+=1
  return pred_labels


def get_res(inferenced_model,test_text):
  texts = []
  
  for sent in test_text:
    texts.append({'text':sent})
  
  result = inferenced_model.inference_from_dicts(dicts=texts) #return_json=True, ,multiprocessing_chunksize=5
  # print("Done with results !! \n")
  # pred_labels = np.zeros(shape=(len(texts),2))
  # temp = np.zeros(shape=(1,2))
  # i=0
  return result



In [ ]:
%%capture
import pandas as pd 
saved_dir1 = '/content/drive/MyDrive/constraint2021/data/siva/saved_models/final_non_augmented_bin_classification_model_xlmr_large_finetuned' #binary
# saved_dir1 = '/content/drive/MyDrive/constraint2021/data/siva/saved_models/multi_42-xlm-roberta-large-2e-5-50epochs_patience7'  #MULTI LABEL 
saved_dir2 = '/content/drive/MyDrive/constraint2021/data/siva/saved_models/final_hierarchial_classification_model_xlmr_large_finetuned'
inferenced_model1 = Inferencer.load(saved_dir1,gpu=True,task_type='text_classification')
inferenced_model2 = Inferencer.load(saved_dir2,gpu=True,task_type='text_classification') 

12/23/2020 07:05:38 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
12/23/2020 07:06:30 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
12/23/2020 07:06:30 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
12/23/2020 07:06:30 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [1024, 2]
12/23/2020 07:06:30 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [1.0694289207458496, 0.9390363693237305]
12/23/2020 07:06:30 - INFO - farm.modeling.prediction_head -   Loading prediction head from /content/drive/MyDrive/constraint2021/data/siva/saved_models/final_non_augmented_bin_classification_model_xlmr_large_finetuned/prediction_head_0.bin
12/23/2020 07:06:46 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'XLMRobertaTokenizer'
12/23/2020 07:06:47 - INF

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ADDED HERE, FINE WITH BINARY CLASSIFICATION AND MULTI-LABEL CLASIFCIATION(yes, giving multiple labels)

basic_texts = [
        {"text": "ये देखो इस्लाम क्या क्या सिखाता है जिहाद से लेकर आतंकवादी और दंगों से लेकर चोरी बुर्खे की आड़ में चद्दर चुराती महिलाएं "},#hate,
        #fake defamation
        {"text":"मोहतरमा  की  साल की छात्रा हैं , और कमाल की बात है कि उनकी बेटी मोना भी में ही पड़ती है"},
        {"text" : "जब इन दलितों को (सभी नहीं) हिन्दू धर्म और हिन्दू देवी देवताओं से इतनी नफरत भारी हुई है तो धूर्त कहीं के अपना नाम हिन्दुओं के जैसे ही क्यों रखते हैं। किसने रोका है कुछ भी बन से, बन जाओ मुस्लिम, ईसाई और जो मन करे। इस धूर्त की हिम्मत नहीं कि किसी दूसरे धर्म के बारे ऐसा बोल दे । "},
        #hate, offensive
        {"text": "डॉक्टर कफ़ील ख़ान को हाईकोर्ट से मिली ज़मानत "},#non-hostile
    ]
    # model = Inferencer.load(save_dir)
result = inferenced_model1.inference_from_dicts(dicts=basic_texts)
result



In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/constraint2021/data/farm_test_cleaned.tsv',sep='\t')
test

,Unique ID,text
0,1,कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नह...
1,2,पटना BMP कैंप में पुरुष और महिला कांस्टेबल ने ...
2,3,"कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊ..."
3,4,अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वा...
4,5,इन पंचर छापों को कोन समझाए कि उनके रोजगार म...
...,...,...
1648,1649,सोनू सूद से प्रेरणा लेकर आदिवासियों ने वो किया...
1649,1650,उमर खालिद पर तो UAPA लगा दिया कपिल मिश्रा क...
1650,1651,पप्पू कमीना
1651,1652,मदर टेरेसा ने अपना पूरा जीवन पीड़ितों की सेवा ...


In [ ]:
from sklearn.metrics import classification_report
from collections import Counter

# Counter({1: 7929, 0: 4071}) TRUSTABLE ?? MAY BE //..
# length = 20#len(test)
length = len(test)


# pred_labels = get_pred_labels(length,inferenced_model1,test)

pred_labels1 = get_pred_labels(length,inferenced_model1,test)

pred_labels2 = get_pred_labels(length,inferenced_model2,test)


# print(pred_labels)

print(Counter(pred_labels1))
print(Counter(pred_labels2))

# actualabels = test['multi_label']

# df = df.assign(predicted_task1 = pred_labels )
# print(classification_report(y_true = actualabels,y_pred = pred_labels))


1653


Inferencing Samples: 100%|██████████| 83/83 [00:16<00:00,  5.15 Batches/s]


1653


Inferencing Samples: 100%|██████████| 83/83 [00:16<00:00,  5.10 Batches/s]

Counter({'not': 852, 'host': 801})
Counter({"['fake']": 1075, "['hate', 'defamation', 'offensive']": 319, "['hate', 'offensive']": 83, "['hate', 'fake']": 83, "['hate', 'fake', 'defamation', 'offensive']": 28, "['hate']": 25, "['hate', 'fake', 'defamation']": 11, "['fake', 'defamation']": 9, "['fake', 'offensive']": 7, "['hate', 'fake', 'offensive']": 4, "['hate', 'defamation']": 4, "['offensive']": 3, '[]': 2})


In [ ]:
predictions = list()
for i in range(len(pred_labels1)):
    if(pred_labels1[i]=='not'):
        predictions.append("['non-hostile']")
    else:
        predictions.append(pred_labels2[i])

In [ ]:
# pred_labels
# len(predictions)
# pred_labels1
# predictions

In [ ]:
# may be the problem is with  blanks , YES THE FIRST NAN IS CAUSING ISSUES 
Will the results improve if somehting chanegd in the classifier ?(FOR THOSE non-empty ones or can i put a differnet goa for training ?)
Trying to set thershold = 0.4 IN PREDICTIONHEAD that may yoedl more.. YES, YOU CAN'T SIMPY GET THE HIGHEST(CAN DO INCASE THERE IS NONE IN PREDOCTED )

# Final Count for test 

Counter({'not': 892, 'host': 761})
Counter({"['fake']": 1075, "['hate', 'defamation', 'offensive']": 319, "['hate', 'offensive']": 83, "['hate', 'fake']": 83, "['hate', 'fake', 'defamation', 'offensive']": 28, "['hate']": 25, "['hate', 'fake', 'defamation']": 11, "['fake', 'defamation']": 9, "['fake', 'offensive']": 7, "['hate', 'fake', 'offensive']": 4, "['hate', 'defamation']": 4, "['offensive']": 3, '[]': 2})


##Preprocess_labels

In [ ]:
# predlabels = predictions
predlabels = [x[2:-2] for x in predictions]
for i in range(len(predlabels)):
    # print(type(predlabels[i]))
    # if(len(predlabels[i])>1):
    if(predlabels[i].count(',')>0):
        predlabels[i] = "'" + predlabels[i] + "'"
        predlabels[i] = predlabels[i].replace("'","").split(',')
        predlabels[i] = ','.join(predlabels[i])
        # print(predlabels[i])
        # predlabels[i] = predlabels[i][1:-1]
predlabels
#fill out empty ???
# "" problem would be solved after lthat insertion.. hopefuly 

['hate, defamation, offensive',
 'non-hostile',
 'fake',
 'fake',
 'hate, defamation, offensive',
 'fake',
 'non-hostile',
 'hate, offensive',
 'non-hostile',
 'hate, defamation, offensive',
 'fake',
 'hate, defamation, offensive',
 'non-hostile',
 'hate, offensive',
 'hate, defamation, offensive',
 'non-hostile',
 'fake',
 'non-hostile',
 'hate, defamation, offensive',
 'hate, defamation, offensive',
 'non-hostile',
 'non-hostile',
 'non-hostile',
 'fake',
 'non-hostile',
 'hate, offensive',
 'hate, fake',
 'hate, defamation, offensive',
 'hate, fake, defamation, offensive',
 'hate, defamation, offensive',
 'non-hostile',
 'non-hostile',
 'fake, defamation',
 'fake',
 'non-hostile',
 'fake',
 'non-hostile',
 'non-hostile',
 'hate',
 'non-hostile',
 'hate, fake, defamation, offensive',
 'non-hostile',
 'non-hostile',
 'hate, defamation, offensive',
 'non-hostile',
 'fake',
 'non-hostile',
 'fake',
 'non-hostile',
 'non-hostile',
 'hate, defamation, offensive',
 'fake',
 'fake',
 'hate,

## Assign non-hostile to empty

In [ ]:
# yundefned metric problem again, any issus iwrh chanehs nu peo  l w?

##Save_Labels

In [ ]:
import pandas as pd
# df = pd.DataFrame(columns=['Unique ID', 'Labels Set'])
df = pd.DataFrame(index=None)
# df = df.assign({'Unique ID':[i for i in range(1,len(test)+1)] } )
df = df.assign(Unique_ID=[i for i in range(1,len(test)+1)])
# df = df.assign({ 'Labels Set':predlabels} )
df = df.assign(Labels_Set = predlabels )

# df = df.append([[i for i in range(1,len(test)+1)],predlabels])
# df
df=df.rename(columns={'Unique_ID':'Unique ID','Labels_Set':'Labels Set' })
df
df.to_csv('/content/drive/MyDrive/constraint2021/data/siva/siva_unaugmented_test_preds.csv',index=False)

In [ ]:
gt      = "/content/drive/MyDrive/constraint2021/data/siva/siva_unaugmented_test_preds.csv"
d=pd.read_csv(gt)
print(d.isna().sum())
d.fillna(value='non-hostile',inplace=True)
d.to_csv('/content/drive/MyDrive/constraint2021/data/siva/siva_unaugmented_test_preds_nan_replaced.csv',index=False)

Unique ID     0
Labels Set    2
dtype: int64


In [ ]:
d[167:]

,Unique ID,Labels Set
167,168,non-hostile
168,169,fake
169,170,"hate, defamation, offensive"
170,171,"hate, defamation, offensive"
171,172,offensive
...,...,...
1648,1649,non-hostile
1649,1650,"hate, offensive"
1650,1651,offensive
1651,1652,non-hostile


### FOR THINGS

In [ ]:
list(pred_labels)
pred_labels[1][2:-2]

"hate', 'defamation', 'offensive"

In [ ]:
actualabels[:20]

In [ ]:
test = test.assign(predicted_task2 = pred_labels)
test

In [ ]:
test.to_csv('/content/drive/My Drive/ML/Competitions/firehasocmain/data_updated/preprocessed_hindi_test_1509.csv',index=False)

In [ ]:
path = '/content/drive/MyDrive/constraint2021/data/final submissions/Siva_Alfred_3.csv'
df = pd.read_csv(path)
df
df=df.rename(columns={'Unique ID':'Unique ID','pred':'Labels Set' })
df
df.to_csv(path,index=False)

In [ ]:
path = '/content/drive/MyDrive/constraint2021/data/final submissions/Siva_Alfred_3.csv'
df = pd.read_csv(path)
df

,Unique ID,Labels Set
0,1,"hate,offensive,defamation"
1,2,non-hostile
2,3,non-hostile
3,4,"hate,fake,defamation"
4,5,non-hostile
...,...,...
1648,1649,non-hostile
1649,1650,non-hostile
1650,1651,offensive
1651,1652,non-hostile


##Official Validation Script

In [ ]:
#linking drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#imports
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install tweet-preprocessor -q
!pip install emoji -q

import warnings
import torch
import numpy as np
!pip install flair
from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence
import numpy as np
import pandas as pd
import re
import io
import torch
from sklearn.metrics import f1_score,recall_score,precision_score
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
import pandas as pd
import warnings
import preprocessor as p
import emoji
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
import sklearn



warnings.filterwarnings("ignore")


In [ ]:
punctuation_with_exceptions = list(string.punctuation)
punctuation_with_exceptions = [chr  for chr in punctuation_with_exceptions if(chr!='.' and chr!=',')]


p.set_options(p.OPT.NUMBER,p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG)

# Available Options: Option Name	Option Short Code URL	p.OPT.URL Mention	p.OPT.MENTION Hashtag	p.OPT.HASHTAG Reserved Words	p.OPT.RESERVED Emoji	p.OPT.EMOJI Smiley	p.OPT.SMILEY Number	p.OPT.NUMBER

def prepro_tweet(row):
    text_before_prepro = row['text']
    text_before_prepro = p.clean(text_before_prepro)
    return text_before_prepro

def deEmojify(row):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', row['text'])

def remove_RT(row):  # more of these symbols are created by emoji_to_text_before_prepro()
  text_before_prepro = row['text']
  text_before_prepro = text_before_prepro.replace('RT', ' ')
  return text_before_prepro


def punct_except_fullstop_comma(row):
  text_before_prepro = row['text']
  for char in punctuation_with_exceptions:
    text_before_prepro = text_before_prepro.replace(char, '')
  return text_before_prepro
def multiclass(row):
  lab=row['Labels Set'].split(',')
  h=0.0
  o=0.0
  f=0.0
  d=0.0
  if 'hate' in lab:
    h=1.0
  if 'offensive' in lab:
    o=1.0
  if 'fake' in lab:
    f=1.0
  if 'defamation' in lab:
    d=1.0
  if h or o or f or d:
    hs=1.0
  else:
    hs=0.0
  return [h,o,d,f,hs]
def preprocess(df):
  df = df.assign(text = [ sent for sent in df['Post']])
  for i in range(len(classf)-1,-1,-1):
    df.insert(2,classf[i],np.zeros(df.shape[0]))
  df['text'] = df.apply(prepro_tweet, axis=1)
  df['text'] = df.apply(remove_RT, axis=1)
  df['text'] = df.apply(punct_except_fullstop_comma, axis=1)
  df['text'] = df.apply(deEmojify,axis=1)

  return df

In [ ]:

### Order of Labels --> [Hostile,defamation,fake,hate,offensive,non-hostile]
### An example      --> [1,0,1,1,0,0]


import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def preprocess(df):
    
    df = df.dropna()
    
    df.insert(len(df.columns)-1,'Hostile', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Defamation', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Fake', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Hate', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Offensive', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Non-Hostile', np.zeros(len(df),dtype=int))    
    
    for i in range(len(df)):
        text = df['Labels Set'][i]
        text = text.lower()
        text = text.replace('\n',"")
        text = text.replace('"',"")
        text = text.replace(" ","")
        text = text.split(',')


        for word in text:
            if word == 'defamation':
                df.at[i,'Hostile']    = 1
                df.at[i,'Defamation'] = 1
    
            if word == 'fake':
                df.at[i,'Hostile']    = 1
                df.at[i,'Fake'] = 1
    
            if word == 'hate':
                df.at[i,'Hostile']    = 1
                df.at[i,'Hate'] = 1
    
            if word == 'offensive':
                df.at[i,'Hostile']    = 1
                df.at[i,'Offensive'] = 1
    
            if word == 'non-hostile' and df['Hostile'][i]==0:
                df.at[i,'Hostile']    = 0
                df.at[i,'Non-Hostile'] = 1

    return df 




def preprocess2(df):
    
    df = df.dropna()
    
    df.insert(len(df.columns)-1,'Hostile', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Defamation', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Fake', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Hate', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Offensive', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Non-Hostile', np.zeros(len(df),dtype=int))    
    print(df.columns)
    for i in range(len(df)):
        text = df['pred'][i]
        text = text.lower()
        text = text.replace('\n',"")
        text = text.replace('"',"")
        text = text.replace(" ","")
        text = text.split(',')


        for word in text:
            # print(word)
            if word == 'defamation':
                df.at[i,'Hostile']    = 1
                df.at[i,'Defamation'] = 1
    
            if word == 'fake':
                df.at[i,'Hostile']    = 1
                df.at[i,'Fake'] = 1
    
            if word == 'hate':
                df.at[i,'Hostile']    = 1
                df.at[i,'Hate'] = 1
    
            if word == 'offensive':
                df.at[i,'Hostile']    = 1
                df.at[i,'Offensive'] = 1
    
            if word == 'non-hostile' and df['Hostile'][i]==0:
                df.at[i,'Hostile']    = 0
                df.at[i,'Non-Hostile'] = 1

    return df 


def preprocess3(df):
    
    df = df.dropna()
    
    df.insert(len(df.columns)-1,'Hostile', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Defamation', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Fake', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Hate', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Offensive', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Non-Hostile', np.zeros(len(df),dtype=int))    
    print(df.columns)
    for i in range(len(df)):
        text = df['Labels Set'][i]
        text = text.lower()
        text = text.replace('\n',"")
        text = text.replace('"',"")
        text = text.replace(" ","")
        text = text.split(',')


        for word in text:
            # print(word)
            if word == 'defamation':
                df.at[i,'Hostile']    = 1
                df.at[i,'Defamation'] = 1
    
            if word == 'fake':
                df.at[i,'Hostile']    = 1
                df.at[i,'Fake'] = 1
    
            if word == 'hate':
                df.at[i,'Hostile']    = 1
                df.at[i,'Hate'] = 1
    
            if word == 'offensive':
                df.at[i,'Hostile']    = 1
                df.at[i,'Offensive'] = 1
    
            if word == 'non-hostile' and df['Hostile'][i]==0:
                df.at[i,'Hostile']    = 0
                df.at[i,'Non-Hostile'] = 1

    return df 
  
    



def get_scores(y_true, y_pred):
    
    hostility_true = y_true['Hostile']
    hostility_pred = y_pred['Hostile']
    print(hostility_true,hostility_pred)
    hostility_f1 = f1_score(y_true=hostility_true, y_pred=hostility_pred, average='weighted')
    
    fine_true = y_true[['Defamation','Fake','Hate','Offensive']]
    fine_pred = y_pred[['Defamation','Fake','Hate','Offensive']]
    # fine_acc = accuracy_score(y_true=fine_true, y_pred=fine_pred)
    # fine_prec = precision_score(y_true=fine_true, y_pred=fine_pred, average=None)
    # fine_recal = recall_score(y_true=fine_true, y_pred=fine_pred, average=None)

    # print("Coarse Grained acc: ", fine_acc)
    # print("Coarse Grained precision: ", fine_prec)
    # print("Coarse Grained recall: ", fine_recal)

    fine_f1          = f1_score(y_true=fine_true, y_pred=fine_pred, average=None)
    defame_f1        = fine_f1[0]
    fake_f1          = fine_f1[1]
    hate_f1          = fine_f1[2]
    offensive_f1     = fine_f1[3]
    weighted_fine_f1 = f1_score(y_true=fine_true, y_pred=fine_pred, average='weighted')

    return [hostility_f1, defame_f1, fake_f1, hate_f1, offensive_f1, weighted_fine_f1]


submission_file_path = '/content/drive/MyDrive/constraint2021/data/final submissions/Siva_Alfred_1.csv'
# submission_file_path = '/content/drive/MyDrive/constraint2021/data/siva/siva_unaugmented_test_preds_nan_replaced.csv'

# ground_truth_path      = "/content/drive/MyDrive/constraint2021/data/siva/test_ground_truth.csv"
# submission_file_path   = "/content/drive/MyDrive/constraint2021/data/siva/Validation_Predictions.csv"

ground_truth_path = '/content/drive/MyDrive/test_groundtruth.csv'

try:  
    y_true = pd.read_csv(ground_truth_path)
    y_pred = pd.read_csv(submission_file_path)
    # print(y_true,y_pred)
    y_true = preprocess(y_true)
    y_pred = preprocess(y_pred)
    print(len(y_true),len(y_pred),"Lengths of true and pred after pre-processing ")
    team_score = get_scores(y_true,y_pred)

except Exception as e:
    print(e)
    team_score = [0,0,0,0,0,0]
    
        
print("Coarse Grained F1-score: ", team_score[0])
print("Defamation F1-score:     ", team_score[1])
print("Fake F1-score:           ", team_score[2])
print("Hate F1-score:           ", team_score[3])
print("Offensive F1-score:      ", team_score[4])
print("Fine Grained F1-score:   ", team_score[5])


1653 1653 Lengths of true and pred after pre-processing 
0       1
1       0
2       1
3       1
4       1
       ..
1648    0
1649    1
1650    1
1651    0
1652    0
Name: Hostile, Length: 1653, dtype: int64 0       1
1       0
2       1
3       1
4       1
       ..
1648    0
1649    0
1650    1
1651    0
1652    0
Name: Hostile, Length: 1653, dtype: int64
Coarse Grained F1-score:  0.8988826277760222
Defamation F1-score:      0.39215686274509803
Fake F1-score:            0.6074074074074074
Hate F1-score:            0.5380434782608695
Offensive F1-score:       0.5362776025236592
Fine Grained F1-score:    0.5360832141670716


In [ ]:
ground_truth_path      = "/content/drive/My Drive/Projects/Hatespeech/Hindi/constraint2021/data/test_with_labels.xlsx"
# submission_file_path   = "/content/drive/MyDrive/constraint2021/data/siva/Validation_Predictions.csv"
submission_file_path = '/content/drive/My Drive/Projects/Hatespeech/Hindi/constraint2021/XLM_val_pred.csv'
t2=pd.read_csv("/content/drive/My Drive/Projects/Hatespeech/Hindi/constraint2021/data/testval.csv")


y_pred = pd.read_csv(submission_file_path)
# print(y_true,y_pred)
y_true =preprocess2(t2)
y_pred = preprocess2(y_pred)
print(len(y_true),len(y_pred),"Lengths of true and pred after pre-processing ")
team_score = get_scores(y_true,y_pred)

team_score

Index(['Unnamed: 0', 'Unique ID', 'Hostile', 'Defamation', 'Fake', 'Hate',
       'Offensive', 'Non-Hostile', 'pred'],
      dtype='object')
Index(['Unnamed: 0', 'Unique ID', 'Hostile', 'Defamation', 'Fake', 'Hate',
       'Offensive', 'Non-Hostile', 'pred'],
      dtype='object')
1653 1653 Lengths of true and pred after pre-processing 
0       1
1       0
2       1
3       1
4       1
       ..
1648    0
1649    1
1650    1
1651    0
1652    0
Name: Hostile, Length: 1653, dtype: int64 0       1
1       0
2       1
3       0
4       0
       ..
1648    0
1649    0
1650    0
1651    0
1652    0
Name: Hostile, Length: 1653, dtype: int64


[0.6999587042711304,
 0.24581005586592178,
 0.42084168336673344,
 0.3644646924829157,
 0.41739130434782606,
 0.37531009976883456]

In [ ]:
ORIGINAL MODEL, NAN REPLACED WITH NON-HOSTILE T=0.5 NORMAL METRIC

Name: Hostile, Length: 811, dtype: int64
Coarse Grained F1-score:  0.8722473494575971
Defamation F1-score:      0.43093922651933697
Fake F1-score:            0.6275862068965518
Hate F1-score:            0.5387453874538746
Offensive F1-score:       0.540983606557377
Fine Grained F1-score:    0.552398705868385

 MODEL, NAN REPLACED WITH NON-HOSTILE T=0.4 NORMAL METRIC
Coarse Grained F1-score:  0.9088228504886837
Defamation F1-score:      0.45070422535211263
Fake F1-score:            0.620253164556962
Hate F1-score:            0.5586206896551724
Offensive F1-score:       0.5438596491228069
Fine Grained F1-score:    0.5586901142287659


hierarchial non-augmented

Coarse Grained F1-score:  0.9009867505667915
Defamation F1-score:      0.4324324324324324
Fake F1-score:            0.6785714285714286
Hate F1-score:            0.5209003215434083
Offensive F1-score:       0.5431309904153354
Fine Grained F1-score:    0.5669116738917341

Data Augmentation hierarchial

Coarse Grained F1-score:  0.9037650436071754
Defamation F1-score:      0.34013605442176864
Fake F1-score:            0.6735395189003437
Hate F1-score:            0.5136186770428016
Offensive F1-score:       0.4854368932038835
Fine Grained F1-score:    0.5343441193094207

No data aug but 25% as thrshold 
Coarse Grained F1-score:  0.9137158105642826
Defamation F1-score:      0.35955056179775285
Fake F1-score:            0.6819672131147541
Hate F1-score:            0.5251798561151079
Offensive F1-score:       0.5619834710743803
Fine Grained F1-score:    0.5610093978891348
